<a href="https://colab.research.google.com/github/Eviaiy/Sentiment_Analysis/blob/main/Analyse_des_sentiments_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U textblob

In [ ]:
pip install nltk

In [ ]:
#libraries
%matplotlib inline
import pandas as pd
import numpy as np
from textblob import TextBlob
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import pickle as cPickle
from scipy.io import loadmat
from sklearn.svm import SVC
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer,TfidfTransformer
from sklearn.metrics import classification_report, f1_score, confusion_matrix, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, GridSearchCV, learning_curve 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

/content


In [ ]:
# Reading data using pandas
path_to_data = "drive/My Drive/IMDB_Dataset.csv"
df = pd.read_csv(path_to_data)

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Size of df

In [ ]:
print(len(df))

50000


In [ ]:
df.groupby('sentiment').describe()

review                                                            \
           count unique                                                top   
sentiment                                                                    
negative   25000  24698  This show comes up with interesting locations ...   
positive   25000  24884  Loved today's show!!! It was a variety and not...   

                
          freq  
sentiment       
negative     3  
positive     5

In [ ]:
import pandas as pd

Regular expressions are used for text searching and more advanced text manipulation.

In [ ]:
# Reading data using pandas
path_to_data = "drive/My Drive/IMDB_Dataset.csv"
df = pd.read_csv(path_to_data)

# Shuffle and Clip data
import random
sorted(df, key=lambda *args: random.random())
#df = shuffle(df)
df = df[:50000]

# Function to clean text. Remove tagged entities, hyperlinks, emojis
import re
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
 
df['review'] = df['review'].apply(clean_text)

# Function to convert labels to number.
def sentiment2label(sentiment):
    if sentiment == "positive":
        return 1
    else :
        return 0

df['sentiment'] = df['sentiment'].apply(sentiment2label)

# List of class names.
class_names = ['negative', 'positive']

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. br br The filmi...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,Petter Mattei's Love in the Time of Money is a...,1


# **TextBlob** 
TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In [ ]:
def tokens(text):
    return TextBlob(text).words

In [ ]:
#df.head().text.apply(tokens)
df['review'] = df['review'].apply(tokens)

In [ ]:
df.head()

,review,sentiment
0,"[One, of, the, other, reviewers, has, mentione...",1
1,"[A, wonderful, little, production, br, br, The...",1
2,"[I, thought, this, was, a, wonderful, way, to,...",1
3,"[Basically, there, 's, a, family, where, a, li...",0
4,"[Petter, Mattei, 's, Love, in, the, Time, of, ...",1


**Stemming** is faster because it chops words without knowing the context of the word in given sentences. **Lemmatization** is slower as compared to stemming but it **knows the context of the word** before proceeding. It is a rule-based approach.
(Octupi and Octupus example)

In [ ]:
#def lemmatization(text):
  #text=''.join(text)
  #wordss = TextBlob(text.lower()).words
    # for each word, take its "base form" = lemma
  #return [word.lemma for word in wordss]

**Stem may not be an actual word whereas, lemma is an actual language word.**

In [ ]:
def lemmatiza(text):
  # for each word, take its "base form" = lemma 
  return [word.lemmatize().lower() for word in TextBlob(text).words]

In [ ]:
#df['review'] = df['review'].apply(lemmatization)
df['review']= df['review'].apply(lambda x: lemmatiza(' '.join(x)))

In [ ]:
df.head()

,review,sentiment
0,"[one, of, the, other, reviewer, ha, mentioned,...",1
1,"[a, wonderful, little, production, br, br, the...",1
2,"[i, thought, this, wa, a, wonderful, way, to, ...",1
3,"[basically, there, 's, a, family, where, a, li...",0
4,"[petter, mattei, 's, love, in, the, time, of, ...",1


In [ ]:
bow_transformer = CountVectorizer(analyzer=lemmatiza).fit(df['review'].apply(lambda x: ','.join(map(str, x))))
print(len(bow_transformer.vocabulary_))

127129


In [ ]:
review1=df['review'][8]
print(review1)
print(len(review1))

['encouraged', 'by', 'the', 'positive', 'comment', 'about', 'this', 'film', 'on', 'here', 'i', 'wa', 'looking', 'forward', 'to', 'watching', 'this', 'film', 'bad', 'mistake', 'i', "'ve", 'seen', '950', 'film', 'and', 'this', 'is', 'truly', 'one', 'of', 'the', 'worst', 'of', 'them', 'it', "'s", 'awful', 'in', 'almost', 'every', 'way', 'editing', 'pacing', 'storyline', "'acting", 'soundtrack', 'the', 'film', "'s", 'only', 'song', 'a', 'lame', 'country', 'tune', 'is', 'played', 'no', 'le', 'than', 'four', 'time', 'the', 'film', 'look', 'cheap', 'and', 'nasty', 'and', 'is', 'boring', 'in', 'the', 'extreme', 'rarely', 'have', 'i', 'been', 'so', 'happy', 'to', 'see', 'the', 'end', 'credit', 'of', 'a', 'film', 'br', 'br', 'the', 'only', 'thing', 'that', 'prevents', 'me', 'giving', 'this', 'a', '1', 'score', 'is', 'harvey', 'keitel', 'while', 'this', 'is', 'far', 'from', 'his', 'best', 'performance', 'he', 'at', 'least', 'seems', 'to', 'be', 'making', 'a', 'bit', 'of', 'an', 'effort', 'one', '

In [ ]:
def clean(text):
  return [word for word in text.split() if word not in sw_nltk]

In [ ]:
sw_nltk = stopwords.words('english')
df['review']= df['review'].apply(lambda x: clean(' '.join(x)))

In [ ]:
review1=df['review'][8]
print(review1)
print(len(review1))

['encouraged', 'positive', 'comment', 'film', 'wa', 'looking', 'forward', 'watching', 'film', 'bad', 'mistake', "'ve", 'seen', '950', 'film', 'truly', 'one', 'worst', "'s", 'awful', 'almost', 'every', 'way', 'editing', 'pacing', 'storyline', "'acting", 'soundtrack', 'film', "'s", 'song', 'lame', 'country', 'tune', 'played', 'le', 'four', 'time', 'film', 'look', 'cheap', 'nasty', 'boring', 'extreme', 'rarely', 'happy', 'see', 'end', 'credit', 'film', 'br', 'br', 'thing', 'prevents', 'giving', '1', 'score', 'harvey', 'keitel', 'far', 'best', 'performance', 'least', 'seems', 'making', 'bit', 'effort', 'one', 'keitel', 'obsessive']
70


In [ ]:
bow=bow_transformer.transform(review1)
print(bow)
print(bow.shape)

  (0, 39067)	1
  (1, 88579)	1
  (2, 27078)	1
  (3, 43639)	1
  (4, 121334)	1
  (5, 68523)	1
  (6, 45806)	1
  (7, 122285)	1
  (8, 43639)	1
  (9, 14467)	1
  (10, 74660)	1
  (11, 5265)	1
  (12, 99732)	1
  (14, 43639)	1
  (15, 115943)	1
  (16, 81857)	1
  (17, 125013)	1
  (18, 4052)	1
  (19, 14016)	1
  (20, 9321)	1
  (21, 40715)	1
  (22, 122431)	1
  (23, 37911)	1
  (24, 84077)	1
  (25, 106953)	1
  :	:
  (45, 52461)	1
  (46, 99637)	1
  (47, 39090)	1
  (48, 29647)	1
  (49, 43639)	1
  (50, 19506)	1
  (51, 19506)	1
  (52, 112490)	1
  (53, 89435)	1
  (54, 49109)	1
  (55, 5543)	1
  (56, 98951)	1
  (57, 52762)	1
  (58, 63267)	1
  (59, 42473)	1
  (60, 16924)	1
  (61, 85966)	1
  (62, 66406)	1
  (63, 99729)	1
  (64, 70256)	1
  (65, 17659)	1
  (66, 38084)	1
  (67, 81857)	1
  (68, 63267)	1
  (69, 80437)	1
(70, 127129)


In [ ]:
#print(bow_transformer.get_feature_names()[56850])

In [ ]:
review_bow = bow_transformer.transform(df['review'].apply(lambda x: ','.join(map(str, x))))

In [ ]:
print('sparse matrix shape:', review_bow.shape)
print('number of non-zeros:', review_bow.nnz) #learn this
print( 'sparsity: %.2f%%' % (100.0 * review_bow.nnz))

sparse matrix shape: (50000, 127129)
number of non-zeros: 4949594
sparsity: 494959400.00%




shape of sparse matrix n*m where n are total documents and m are total unique words.

CountVectorizer just count the frequency of word in that document. But many time few words such as(the, or) occurs a lot of time in collection which really don't contribute in deciding the polarity of particular document so to nullify their effect special weighting method is to be used.


Tf-idf Vectorizer
Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency.

The goal of using tf-idf instead of the just CountVectorizer in a given document is to scale down the impact of tokens that occur very frequently in a given corpus(which are less informative) than tokens which occur few times.

Tf-idf(d,t)=tf(d,t)*idf(d,t);

where tf(d,t) is term frequency which states how many times word/token t occur in that doucument devided by total no of words in that document.

and idf(d,t)=log[n/(df(d,t))] i.e. total no of documents divided by no of documents containing that word/token t.

In [ ]:
tfidf_transformer =TfidfTransformer().fit(review_bow)
review_tfidf = tfidf_transformer.transform(review_bow)
review_tfidf.shape

(50000, 127129)

In [ ]:
review_train, review_test, sentiment_train, sentiment_test = train_test_split(df['review'].apply(lambda x: ','.join(map(str, x))), df['sentiment'], test_size=0.3)
print(len(review_train), len(review_test), len(sentiment_train) , len(sentiment_test))

35000 15000 35000 15000


In [ ]:
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=lemmatiza)),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC()),
])

In [ ]:
# pipeline parameters to automatically explore and tune
param_svm = [
  {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
  {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]

In [ ]:
grid_svm = GridSearchCV(
    pipeline_svm, #object used to fit the data
    param_grid=param_svm, 
    refit=True,  # fit using all data, on the best detected classifier
    n_jobs=-1,  # number of cores to use for parallelization; -1 for "all cores" i.e. to run on all CPUs
    scoring='accuracy',#optimizing parameter
    cv=StratifiedKFold(n_splits=5),
)

In [ ]:
%time classifier = grid_svm.fit(review_train, sentiment_train) # find the best combination from param_svm
print(classifier.cv_results_)
#print(classifier.gsearch.cv_results_)

In [ ]:
print(classification_report(sentiment_test, classifier.predict(review_test)))

In [ ]:
print(classifier.predict(["I don't like this product it is too expensive and bad quality"])[0])